In [1]:
import pandas as pd
import torch

from sklearn.model_selection import train_test_split
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, \
    TrainingArguments, DataCollatorWithPadding, Trainer
from datasets import load_dataset

model_name = 'bert-base-uncased'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
# n_rows = 1000
# train = pd.read_csv('../data/train.csv')[:n_rows]
# test = pd.read_csv('../data/test.csv')[:n_rows]

# def tokenize_dataset(tokenizer, dataset):
#     return tokenizer(dataset["question_text"].to_list(), padding=True, truncation=True, return_tensors='pt')


tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# train, valid = train_test_split(train, test_size=0.01, stratify=train.target)
# train.index, valid.index = range(train.shape[0]), range(valid.shape[0])

# train_ds = tokenize_dataset(tokenizer, train)
# train_ds['labels'] = train['target']
# train_ds = data_collator(train_ds).to(device)

# valid_ds = tokenize_dataset(tokenizer, valid)
# valid_ds['labels'] = valid['target']
# valid_ds = data_collator(valid_ds).to(device)

# test_ds = tokenize_dataset(tokenizer, test)
# test_ds = data_collator(test_ds).to(device)

dataset = load_dataset('csv', data_files='../data/train.csv')['train'].train_test_split(test_size=15_000)

Found cached dataset csv (/home/olehborysevych/.cache/huggingface/datasets/csv/default-31e0f765990aa99d/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset['train'][0]

{'qid': '254c52dcb529839d515c',
 'question_text': 'Is inheritance tax necessary or is it burdensome?',
 'target': 0}

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

training_args = TrainingArguments(
    output_dir='../models',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    gradient_accumulation_steps=1,
    evaluation_strategy="epoch",
    save_strategy='epoch',
    load_best_model_at_end=True,
    # metric_for_best_model
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=None
)

trainer.train()

In [ ]:
classifier.save_pretrained('../models/bert')